# Hybrid Search Exercise

In this notebook, we will delve into the implementation details of the hybrid search using practical tools in the RAG application. You will have the opportunity to implement it using build-in sparse vector support in a vector database (Qdrant), and a popular reranking model from Cohere.



### Visual improvements

In [4]:
from rich.console import Console
from rich_theme_manager import Theme, ThemeManager
import pathlib

theme_dir = pathlib.Path("../themes")
theme_manager = ThemeManager(theme_dir=theme_dir)
dark = theme_manager.get("dark")

# Create a console with the dark theme
console = Console(theme=dark)

## Hybrid Search - Sparse Index

You will use sparse vector supported database to complement the semantic search (dense vector). You will calculate the sparse and dense vectors using sparse and dense encoders. For the sparse encoder you will use Splade.

### Loading complex documents

You will read a couple of the legal documents, we used in the previous exercise. For simplicity the chunking will be based on the document pages, and not the more time consuming semantic chunking, you used in the previous exercise. 

In [2]:
import pymupdf4llm

import requests
import os

start_doc_number = 193
end_doc_number = 194

doc_urls = [
    f"https://static.case.law/wash-app/{doc_number}.pdf" 
    for doc_number in range(start_doc_number, end_doc_number + 1)
]

md_texts = []  # Initialize an empty list to store md_text for each file
local_data_folder = "data"
import os
if not os.path.exists(local_data_folder):
    os.makedirs(local_data_folder)

for url in doc_urls:
    response = requests.get(url)
    # Download a local copy of the PDF to the local data folder
    local_pdf_path = os.path.join(local_data_folder, url.split('/')[-1])
    with open(local_pdf_path, 'wb') as f:
        f.write(response.content)
    
    # Use pymupdf4llm functions on the local file
    md_text = pymupdf4llm.to_markdown(local_pdf_path, page_chunks=True)
    md_texts.append(md_text)  # Append the md_text to the list

Processing data/193.pdf...
[                                        ] (0/1041[                                        ] (   1/104[                                        ] (   2/104[                                        ] (   3/104[                                        ] (   4/104[                                        ] (   5/104[                                        ] (   6/104[                                        ] (   7/104[                                        ] (   8/104[                                        ] (   9/104[                                        ] (  10/104[                                        ] (  11/104[                                        ] (  12/104[                                        ] (  13/104[                                        ] (  14/104[                                        ] (  15/104[                                        ] (  16/104[                                        ] (  17/104[                                      

### Chunking the documents 

You will create chunks based on the pages of the documents. Let's create the metadata of the chunks with the page number, and document URL

In [3]:
console.print(md_texts[0][100])

{
    'metadata': {
        'format': 'PDF 1.3',
        'title': 'Washington Appellate Reports volume 193',
        'author': 'anonymous',
        'subject': 'unspecified',
        'keywords': '',
        'creator': 'Harvard Library Innovation Lab',
        'producer': 'ReportLab PDF Library - www.reportlab.com',
        'creationDate': "D:20190924084812+00'00'",
        'modDate': "D:20190924084812+00'00'",
        'trapped': '',
        'encryption': None,
        'file_path': 'data/193.pdf',
        'page_count': 1041,
        'page': 101
    },
    'toc_items': [],
    'tables': [],
    'images': [
        {
            'number': 0,
            'bbox': Rect(0.0, 50.0, 396.0, 561.760009765625),
            'transform': (396.0, 0.0, -0.0, 611.760009765625, 0.0, 0.0),
            'width': 1650,
            'height': 2549,
            'colorspace': 1,
            'cs-name': 'DeviceGray',
            'xres': 96,
            'yres': 96,
            'bpc': 1,
            'size': 39013
        }
    ],
    'graphics': [],
    'text': 'used the property with Mr. Deno’s permission. The court\n\nquieted title in the LeBleus, ejected the 
Aalgaards, and\n\nordered them to remove their house, barn, and shed from\n\nthe property within 30 days.\n\n     -
Upon the Aalgaards’ filing of a notice of appeal, the\n\ncourt stayed its order of ejectment.\n\nANALYSIS\n\n      
- 0 To establish a claim of adverse possession, a\n\nparty’s possession of property must be (1) exclusive, 
(2)\n\nactual and uninterrupted, (3) open and notorious, and (4)\n\nhostile and under a claim of right. Chaplin v. 
Sanders, 100\n\nWn.2d 853, 857, 676 P.2d 431 (1984). All of these elements\n\nmust exist concurrently for at least 
10 years. RCW4.16.020.\n\nBecause courts presume that the holder of legal title is in\n\npossession, “the party 
claiming to have adversely possessed\n\nthe property has the burden of establishing the existence of\n\neach 
element.” ITT Rayonier, Inc. v. Bell, 112 Wn.2d 754,\n\n757, 774 P.2d 6 (1989).\n\n       - 1 The only element of 
adverse possession that the\n\nLeBleus claim is not established“ by the Aalgaards is that of”\nhostility. Hostility
‘does not import enmity or ill-will.’\n\nChaplin, 100Wn.2d at 857 (quoting King v. Bassindale, 127\n\nWash. 189, 
192, 220 P. 777 (1923)).\n\nThe “hostility/claim of right” element of adverse possession\n\nrequires only that the 
claimant treat the land as his own as\nagainst the world throughout the statutory period. The nature\n\nof his 
possession will be determined solely on the basis of the\n\nmanner in which he treats the property. His subjective 
belief\n\nregarding his true interest in the land and his intent to\n\ndispossess or not dispossess another is 
irrelevant to this\n\ndetermination.\n\nId. at 860-61.\n\n       - 2 The Aalgaards treated the property as their 
own;\n\nthey constructed a home and other significant improve-\n\n\n-----\n\n',
    'words': []
}

Let's check how many pages are in the first document:

In [ ]:
number_of_pages = ### YOUR CODE HERE ###
console.print(number_of_pages)

### Creating the Sparse Index

We will use an advanced sparse encoder, [SPLADE](https://github.com/naver/splade). This method uses a pre-trained model to tokenize and calcualte the score for each token, as well as expanding tokens, not found in the document. For example, "efficient neural search" will generate the tokens: "efficient", "neural", "bert", and "search", with their relevant weights or scores. 

In [ ]:
from transformers import AutoModelForMaskedLM, AutoTokenizer

model_id = "naver/splade-cocondenser-ensembledistil"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForMaskedLM.from_pretrained(model_id)

### Calculating Sparse Vector

The following function compute the sparse vector of a given text using the Splade encoder.

In [5]:
import torch


def compute_vector(text):
    """
    Computes a vector from logits and attention mask using ReLU, log, and max operations.
    """
    tokens = tokenizer(text, return_tensors="pt")
    output = model(**tokens)
    logits, attention_mask = output.logits, tokens.attention_mask
    relu_log = torch.log(1 + torch.relu(logits))
    weighted_log = relu_log * attention_mask.unsqueeze(-1)
    max_val, _ = torch.max(weighted_log, dim=1)
    vec = max_val.squeeze()

    return vec, tokens

test_text = """
Arthur Robert Ashe Jr. (July 10, 1943 – February 6, 1993) was an American professional tennis player. 
He won three Grand Slam titles in singles and two in doubles.
"""
vec, tokens = compute_vector(test_text)
console.print(vec.shape)

torch.Size([30522])

### Explore Sparse Vector 

The following function calculates the tokens and their weights from the sparse vector and returns a dictionary with textual tokens as keys.

In [6]:
def extract_and_map_sparse_vector(vector, tokenizer):
    """
    Extracts non-zero elements from a given vector and maps these elements to their human-readable tokens using a tokenizer. The function creates and returns a sorted dictionary where keys are the tokens corresponding to non-zero elements in the vector, and values are the weights of these elements, sorted in descending order of weights.

    This function is useful in NLP tasks where you need to understand the significance of different tokens based on a model's output vector. It first identifies non-zero values in the vector, maps them to tokens, and sorts them by weight for better interpretability.

    Args:
    vector (torch.Tensor): A PyTorch tensor from which to extract non-zero elements.
    tokenizer: The tokenizer used for tokenization in the model, providing the mapping from tokens to indices.

    Returns:
    dict: A sorted dictionary mapping human-readable tokens to their corresponding non-zero weights.
    """

    # Extract indices and values of non-zero elements in the vector
    cols = vector.nonzero().squeeze().cpu().tolist()
    weights = vector[cols].cpu().tolist()

    # Map indices to tokens and create a dictionary
    idx2token = {idx: token for token, idx in tokenizer.get_vocab().items()}
    token_weight_dict = {
        idx2token[idx]: round(weight, 2) for idx, weight in zip(cols, weights)
    }

    # Sort the dictionary by weights in descending order
    sorted_token_weight_dict = {
        k: v
        for k, v in sorted(
            token_weight_dict.items(), key=lambda item: item[1], reverse=True
        )
    }

    return sorted_token_weight_dict


# Usage example
sorted_tokens = extract_and_map_sparse_vector(vec, tokenizer)
console.print(sorted_tokens)

{
    'ashe': 2.95,
    'arthur': 2.61,
    'tennis': 2.22,
    'robert': 1.74,
    'jr': 1.55,
    'he': 1.39,
    'founder': 1.36,
    'doubles': 1.24,
    'won': 1.22,
    'slam': 1.22,
    'died': 1.19,
    'singles': 1.1,
    'was': 1.07,
    'player': 1.06,
    'titles': 0.99,
    'birthday': 0.99,
    'grand': 0.93,
    'champion': 0.93,
    'many': 0.92,
    'actor': 0.92,
    'his': 0.89,
    'death': 0.89,
    '1943': 0.86,
    'merlin': 0.86,
    'birth': 0.85,
    'date': 0.84,
    'win': 0.83,
    'professional': 0.81,
    '1993': 0.79,
    'early': 0.78,
    'born': 0.77,
    'medal': 0.75,
    'double': 0.73,
    'sr': 0.73,
    'championship': 0.71,
    'played': 0.59,
    'celebrity': 0.59,
    'nationality': 0.59,
    'wimbledon': 0.53,
    'championships': 0.5,
    'williams': 0.5,
    'son': 0.49,
    'carter': 0.49,
    'tournament': 0.44,
    'title': 0.43,
    '1991': 0.43,
    'champ': 0.43,
    'edgar': 0.4,
    'alex': 0.38,
    'medals': 0.37,
    'him': 0.35,
    'career': 0.35,
    'hale': 0.33,
    'bob': 0.32,
    'nathan': 0.31,
    'players': 0.3,
    'retired': 0.29,
    'slams': 0.29,
    'award': 0.26,
    'gilbert': 0.26,
    'had': 0.25,
    'bruce': 0.25,
    'single': 0.23,
    'play': 0.23,
    'andre': 0.22,
    'wins': 0.21,
    'finals': 0.2,
    'doug': 0.2,
    'jay': 0.18,
    'three': 0.17,
    'record': 0.16,
    'winning': 0.16,
    'seven': 0.15,
    'fl': 0.14,
    'rob': 0.13,
    'father': 0.12,
    'chess': 0.12,
    'athlete': 0.12,
    'july': 0.1,
    'open': 0.1,
    'bailey': 0.1,
    'serena': 0.09,
    'ali': 0.08,
    'profile': 0.07,
    'tiger': 0.07,
    'held': 0.06,
    'ellis': 0.06,
    'golf': 0.05,
    'haley': 0.05,
    'name': 0.04,
    'jul': 0.04,
    'king': 0.03,
    'morris': 0.03,
    'american': 0.02,
    'alexander': 0.02,
    'wright': 0.02,
    '10': 0.01,
    'charles': 0.01,
    'artist': 0.01,
    'israel': 0.01,
    'davis': 0.01,
    'hawkins': 0.0
}

In [ ]:
len(sorted_tokens)

### Dense Encoder

You will use an encoder that was pre-trained on legal documents.

In [8]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
from openai import OpenAI
import openai
embedding_client = OpenAI()

dense_vector_dimension = 1024
def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return embedding_client.embeddings.create(
            input = [text], 
            model=model,
            # Reduce the size of the embedding vector to 1024
            ### YOUR CODE HERE ###
        ).data[0].embedding


### Create the vector collection



In [10]:
from qdrant_client import QdrantClient
from qdrant_client.http import models

# Qdrant client setup
vector_db_client = QdrantClient(":memory:")

# Define collection name
COLLECTION_NAME = "sparse_legal"

In [11]:
vector_db_client.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config={
        "text-dense": models.VectorParams(
            size=dense_vector_dimension,  # we will compress OpenAI Embeddings
            distance=models.Distance.COSINE,
        )
    },
    sparse_vectors_config={
        "text-sparse": models.SparseVectorParams(
            index=models.SparseIndexParams(
                on_disk=False,
            )
        )
    },
)

True

In [13]:
import uuid

def generate_uuid(doc_url, i):
    return str(uuid.uuid5(uuid.NAMESPACE_URL, f"{doc_url}/{i}"))

from tqdm import tqdm

chunk_num = sum(len(md_text) for md_text in md_texts)
state = "Washington"

corpus_json = []
for index, md_text in enumerate(md_texts):
    doc_url = doc_urls[index]
    for i, chunk in tqdm(enumerate(md_text), total=len(md_text), desc=f"Processing {index+1}/{len(md_texts)} documents"):
        chunk_text = chunk['text']
        title = chunk['metadata']['title']
        # Extract the page number from the metadata
        page = chunk['metadata']['page']
        chunk_uuid = generate_uuid(doc_url, page)
        enriched_chunk = f"{title} \n {chunk_text}"
        sparse_vector, tokens = compute_vector([enriched_chunk[:512]])
        doc_indices = sparse_vector.nonzero().numpy().flatten()
        doc_values = sparse_vector.detach().numpy()[doc_indices]
        vector_db_client.upsert(
            collection_name=COLLECTION_NAME,
            points=[
                models.PointStruct(
                    id=chunk_uuid,
                    payload={
                        "document": chunk_text,
                            "metadata" : {
                            "title": title,
                            "state": state,
                            "doc_url": doc_url,
                            "page": page,
                        }
                    }, 
                    vector={
                        "text-dense": ### YOUR CODE HERE ###
                        "text-sparse": models.SparseVector(
                            indices=doc_indices.tolist(), values=doc_values.tolist()
                        )
                    },
                )
            ],
        )


Processing 2/2 documents: 100%|██████████| 1048/1048 [07:47<00:00,  2.24it/s]


### Searching the Sparse Index

As we are doing in the dense index, we need to tokenize and encode the query text:

In [18]:
# Preparing a query vector

query_text = "Please find me cases that are talking about bank foreclosure of mortage defaults"
query_vec, query_tokens = compute_vector(query_text)

query_indices = query_vec.nonzero().numpy().flatten()
query_values = query_vec.detach().numpy()[query_indices]

console.print(extract_and_map_sparse_vector(query_vec, tokenizer))

{
    'default': 2.17,
    'mort': 2.01,
    'bank': 1.52,
    'fore': 1.5,
    'cases': 1.41,
    'case': 1.31,
    '##ure': 0.91,
    'find': 0.85,
    '##age': 0.82,
    '##cl': 0.72,
    '##os': 0.71,
    'banks': 0.71,
    'talk': 0.65,
    'penalty': 0.64,
    'fraud': 0.58,
    'lawsuit': 0.56,
    'bankruptcy': 0.55,
    'cancel': 0.54,
    'loan': 0.53,
    'error': 0.38,
    'instances': 0.36,
    'legal': 0.35,
    'compromise': 0.34,
    'yes': 0.33,
    'banking': 0.33,
    'talking': 0.31,
    'failure': 0.3,
    'of': 0.29,
    'restoration': 0.29,
    'landlord': 0.29,
    'me': 0.28,
    '##s': 0.27,
    'say': 0.27,
    'issue': 0.26,
    'search': 0.26,
    'clause': 0.26,
    '##iction': 0.26,
    'examples': 0.23,
    'debt': 0.23,
    'mention': 0.22,
    'trustee': 0.22,
    'pre': 0.21,
    'judge': 0.2,
    'list': 0.19,
    'ruin': 0.17,
    'violation': 0.16,
    'court': 0.13,
    'sale': 0.13,
    'lawyer': 0.13,
    'said': 0.12,
    'issues': 0.12,
    'failed': 0.12,
    'jurisdiction': 0.11,
    'similar': 0.1,
    'please': 0.1,
    'types': 0.1,
    'negligence': 0.1,
    'related': 0.08,
    'estate': 0.08,
    'bond': 0.08,
    'divorce': 0.08,
    'rat': 0.08,
    'payment': 0.06,
    'lie': 0.05,
    '##ancy': 0.05,
    'property': 0.04,
    'guarantee': 0.04,
    'same': 0.03,
    'problem': 0.03,
    'situations': 0.02,
    'situation': 0.01
}

And use the encoded query to search the sparse index:

In [17]:
# Searching for similar documents
result = vector_db_client.search(
    collection_name=COLLECTION_NAME,
    query_vector=models.NamedSparseVector(
        name="text-sparse",
        vector=models.SparseVector(
            indices=query_indices,
            values=query_values,
        ),
    ),
    with_vectors=True,
)

result

[ScoredPoint(id='37417cdc-3b52-59c6-9080-9f659f69e69e', version=0, score=12.318807601928711, payload={'document': 'tators’ discussions about the discharge ofdeeds oftrust and\n\nmortgages do not support this result.11\n\n      - 1 Rather, the trial court’s ruling fails to recognize that\n\nenforcement of a promissory note and foreclosure of a deed\n\nof trust securing that note are separate remedies of a\n\ncreditor in the event of a borrower’s default.12 The inability\n\nto pursue one remedy does not bar the other.\n\n     - 2 The trial court’s ruling in this case has a practical\n\neffect. That effect is that the Edmundsons retain ownership\n\nof property without repaying the loan used to purchase it.\n\nThe loss shifts to the lender because the Edmundsons no\nlonger have any personal obligation on the promissory note\n\ndue to the discharge in bankruptcy. Under the trial court’s\n\nruling, the lender also has no right to realize on the\n\ncollateral for the loan. Neither the equity 

### Hybrid Search of Sparse and Dense vectors

You will now search both indexes using `search_batch`. Please note to use the correct encoders for each index.

In [19]:
# Compute the dense embedding vector of the query
query_dense_vector = get_embedding(query_text)

hits = vector_db_client.search_batch(
    collection_name=COLLECTION_NAME,
    requests=[
        models.SearchRequest(
            vector=models.NamedVector(
                name="text-dense",
                vector=query_dense_vector,
            ),
            limit=10,
        ),
        models.SearchRequest(
            vector=models.NamedSparseVector(
                name="text-sparse",
                vector=models.SparseVector(
                    indices=query_indices,
                    values=query_values,
                ),
            ),
            limit=10,
        ),
    ],
)

Create a set (remove duplicates) of chunk ids from the two set of results.

In [20]:
unique_ids = set([hit.id for hit in hits[0] + hits[1]])

len(unique_ids)

16

In [21]:
result_chunks = vector_db_client.retrieve(
    collection_name=COLLECTION_NAME,
    ids=list(unique_ids)
)

In [22]:
console.print(result_chunks[0])

Record(
    id='6002e3e5-314d-54b1-89de-727ce4ac6362',
    payload={
        'document': 'but it also stated that Umpqua’s acceptance of proceeds\n\n“shall not constitute a waiver or 
cure of the defaults under\n\nthe Deed of Trust nor a defense to any sale, or judicial or\n\nnonjudicial 
foreclosure of the Deed ofTrust.” CP at 103-04.\n\nShasta did not object to the receiver’s appointment or 
other\n\nterms in the court’s order.7\n\n     - 1 On May 25, 2012, the court entered a default order\n\nagainst 
Johnson and the other guarantor, Callaway, based\n\non their failure to appear and Shasta’s default on the 
loan\n\nthat they had guaranteed. Between March 19 and May 25,\n\nneither Shasta, Johnson, nor Callaway objected or
re-\n\nsponded to any of Umpqua’s pleadings or the receiver’s\n\nmotions, notices of compensation, or reports.\n\n 
- 2 On July 26, 2013, the receiver moved to have the\n\nsuperior court approve (1) the sale of Shasta’s real 
property\n\n“free and clear of liens,” (2) the payment to the real estate\n\nbroker, and (3) the “assumption and 
assignment of leases.”\n\nCP at 198. The motion was supported by the receiver’s\n\ndeclaration, in which he stated 
that the sale of the real\n\nproperty was “in the best interest of the receivership\n\nestate.” CP at 212. The 
receiver also moved for an exemp-\ntion from the real estate excise tax under RCW 82.45-\n.010(3)(i) because “this 
is a court-ordered sale in an action\n\nwith a pending claim for judicial foreclosure.” CP at 212.\n\n     - 3 The 
superior court granted the receiver’s motions\n\nand approved the sale of the property to a third party 
for\n\n$550,000 on August 9, 2013. The superior court’s order\n\nstated that the sale of the property “shall be 
free and clear\n\nof any and all hens . . . and of all rights of redemption.” CP\n\nat 237. Further, the superior 
court’s order stated “that the\n\nsale ofthe [p]roperty shall be considered an order of sale by\n\nthe [c]ourt to 
execute upon a judgment for the purposes of\n\nRCW 82.45.010(3)0,” exempting the sale from real estate\n\nexcise 
taxes. CP at 238. Neither Shasta, Johnson, nor Calla-\n\n7\nNeither Johnson nor Callaway, the guarantors, ever 
objected to the superior\n\ncourt’s orders, and neither entered a notice of appearance in response to 
Umpqua’s\n\n\n-----\n\n',
        'metadata': {
            'title': 'Washington Appellate Reports volume 194',
            'state': 'Washington',
            'doc_url': 'https://static.case.law/wash-app/194.pdf',
            'page': 721
        }
    },
    vector=None,
    shard_key=None,
    order_value=None
)

### Chunk augmentation

Since the page breaks are arbitrary, you want to concatenate the previous and next page for each of the pages in the results. You will need to use the function to generate the UUID of the chunk in the database, with the metadata of each result.

In [23]:
for result in result_chunks:
    doc_title = result.payload['metadata']['title']
    doc_url = result.payload['metadata']['doc_url']
    page = result.payload['metadata']['page']
    previous_page_uuid = generate_uuid(doc_url, page-1)
    next_page_uuid = generate_uuid(doc_url, page+1)
    
    previous_page_result = vector_db_client.retrieve(
        collection_name=COLLECTION_NAME,
        ids=[previous_page_uuid]
    )
    next_page_result = vector_db_client.retrieve(
        collection_name=COLLECTION_NAME,
        ids=[next_page_uuid]
    )
    
    if previous_page_result:
        previous_page_text = f"<page-break-{page-1}> {previous_page_result[0].payload['document']}"
    else:
        previous_page_text = ""
        
    if next_page_result:
        next_page_text = f"<page-break-{page+1}> {next_page_result[0].payload['document']}"
    else:
        next_page_text = ""
    
    result.payload['document'] = f"""
        Document title: {title}, Document URL: {doc_url}
        {previous_page_text} 
        <page-break-{page}> {result.payload['document']} 
        {next_page_text}"""

In [24]:
console.print(result_chunks[0])

Record(
    id='6002e3e5-314d-54b1-89de-727ce4ac6362',
    payload={
        'document': '\n        Document title: Washington Appellate Reports volume 194, Document URL: 
https://static.case.law/wash-app/194.pdf\n        <page-break-720> into receivership. Umpqua acquired certain 
assets of Ever-\n\ngreen, including the Shasta loan.4\n\n     - After Umpqua’s acquisition of the Shasta 
loan,\n\nShasta modified the loan again in 2010 and 2011. For the\n\nlast loan modification, in January 2011, 
Umpqua and\n\nShasta executed a change in terms agreement modifying\n\nthe amount due under a new replacement note,
and ex-\n\necuted a business loan agreement and commercial security\n\nagreement as additional collateral. The 
commercial secu-\nrity agreement provided Umpqua the right to appointment\n\nof a receiver under chapter 7.60 
RCW.5\n\n     - After August 18, 2011, Shasta failed to make any\n\nfurther payments on its loan, and Umpqua issued
a notice\n\nof default in November. The principal loan balance at the\n\ntime Shasta stopped making payments was 
$1,044,365.27,\n\nincluding accrued interest.\n\nII. Procedural History\n\n     - In March 2012, Umpqua petitioned 
the superior court\n\nto appoint a general receiver under chapter 7.60 RCW and\n\nfor judicial foreclosure under 
the DTA on the Shasta DOT.6\n\nUmpqua’s petition included in its request for relief that\n\nUmpqua “have judgment 
for any deficiency.” CP at 10.\n\n     - 0 In April 2012, the superior court appointed a general\n\nreceiver for 
Shasta and gave the receiver authority to sell\n\nShasta’s property “free and clear ofhens and of all rights 
of\n\nredemption.” CP at 101. The superior court ordered that\n\nUmpqua apply all proceeds to the “reduction of the
loan,”\n\n4\nShasta does not dispute that Umpqua is the owner and holder of the\n\npromissory note, replacement 
notes, and security instruments issued by Shasta,\n\nCallaway, and Johnson.\n\n5 Callaway and Johnson also executed
additional commercial security agree-\n\nments and financing statements in security of the modification. Johnson 
executed\n\na second security agreement in January 2011 in favor of Umpqua.\n\n6 Shasta and Johnson incorrectly 
state that the petition sought to “sell the\n\n\n-----\n\n \n        <page-break-721> but it also stated that 
Umpqua’s acceptance of proceeds\n\n“shall not constitute a waiver or cure of the defaults under\n\nthe Deed of 
Trust nor a defense to any sale, or judicial or\n\nnonjudicial foreclosure of the Deed ofTrust.” CP at 
103-04.\n\nShasta did not object to the receiver’s appointment or other\n\nterms in the court’s order.7\n\n     - 1
On May 25, 2012, the court entered a default order\n\nagainst Johnson and the other guarantor, Callaway, 
based\n\non their failure to appear and Shasta’s default on the loan\n\nthat they had guaranteed. Between March 19 
and May 25,\n\nneither Shasta, Johnson, nor Callaway objected or re-\n\nsponded to any of Umpqua’s pleadings or the
receiver’s\n\nmotions, notices of compensation, or reports.\n\n     - 2 On July 26, 2013, the receiver moved to 
have the\n\nsuperior court approve (1) the sale of Shasta’s real property\n\n“free and clear of liens,” (2) the 
payment to the real estate\n\nbroker, and (3) the “assumption and assignment of leases.”\n\nCP at 198. The motion 
was supported by the receiver’s\n\ndeclaration, in which he stated that the sale of the real\n\nproperty was “in 
the best interest of the receivership\n\nestate.” CP at 212. The receiver also moved for an exemp-\ntion from the 
real estate excise tax under RCW 82.45-\n.010(3)(i) because “this is a court-ordered sale in an action\n\nwith a 
pending claim for judicial foreclosure.” CP at 212.\n\n     - 3 The superior court granted the receiver’s 
motions\n\nand approved the sale of the property to a third party for\n\n$550,000 on August 9, 2013. The superior 
court’s order\n\nstated that the sale of the property “shall be free and clear\n\nof any and all hens .

## Hybrid Search - Merging Results by Reranking

You will use a popular [reranking service by Cohere](https://docs.cohere.com/reference/rerank)

In [25]:
import cohere

co = cohere.Client()

docs = ### YOUR CODE HERE ###

rerank_response = co.rerank(
    model="rerank-english-v3.0",
    query=query_text,
    documents=docs,
    top_n=3,
)

## Using reranked results to generate a reply

We can now take the reranked results and call the LLM to generate the reply to the user's query.

In [32]:
extracted_documents = [
    result_chunks[result.index].payload['document'] 
    for result in rerank_response.results
]

In [33]:
# Now time to connect to the large language model
from openai import OpenAI
from rich.text import Text

client = OpenAI()
system_message = """
You are a paralegal specialist. 
Your top priority is to help lawyers find information in a large legal corpus and help them with their requests. 
Please include citation to the document and the page number.
"""
completion = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": query_text},
        {"role": "assistant", "content": str(extracted_documents)}
    ]
)

In [34]:
from rich.panel import Panel

response_text = Text(completion.choices[0].message.content)
styled_panel = Panel(
    response_text,
    title=f"Reply to '{query_text}'",
    expand=False,
    border_style="bright_yellow",
    padding=(1, 1)
)

console.print(styled_panel)

╭────────── Reply to 'Please find me cases that are talking about bank foreclosure of mortage defaults' ──────────╮
│                                                                                                                 │
│ 1. "Deutsche Bank Nat'l Tr.Co. v. Slotke" talks about how a security instrument follows the note it secures and │
│ it's included in the Washington Appellate Reports volume 194, page 259. You can read it                         │
│ [here](https://static.case.law/wash-app/193.pdf)                                                                │
│                                                                                                                 │
│ 2.  The case "Bain v.Metro. Mortg. Grp., Inc.", mentioned in Washington Appellate Reports volume 194, page 257  │
│ also discusses the topic, stating that if a borrower defaults, a lender may nonjudicially foreclose by a        │
│ trustee’s sale. The document is available [here](https://static.case.law/wash-app/193.pdf)                      │
│                                                                                                                 │
│ 3. The case "Albice v. Premier Mortg. Servs.", discussed in Washington Appellate Reports volume 194, page 257,  │
│ also discusses the goals of the Deeds of Trust Act in the context of foreclosure. The document is available     │
│ [here](https://static.case.law/wash-app/193.pdf)                                                                │
│                                                                                                                 │
│ 4. Lastly, the case "Johnson v. Home State Bank" outlines the relationship between a deed of trust and a        │
│ discharge of debt in bankruptcy. This is found in Washington Appellate Reports volume 194, page 955. The        │
│ document is available [here](https://static.case.law/wash-app/194.pdf)                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯